## Install

In [ ]:

# ! pip install --user --quiet google-cloud-aiplatform==1.25.0
# ! pip install --user --quiet pandas
# ! pip install --user --quiet datasets
# ! pip install --user --quiet google-api-python-client
# ! pip install --user --quiet pypdf
# ! pip install --user --quiet faiss-cpu

In [ ]:
# ! pip install --quiet google-cloud-aiplatform langchain==0.0.229 chromadb==0.3.26 pydantic==1.10.8 typing-inspect==0.8.0 typing_extensions==4.5.0 pandas datasets google-api-python-client pypdf faiss-cpu transformers config --upgrade --user

In [ ]:
!pip install --user SQLAlchemy==2.0 --quiet
!pip install --user sqlalchemy-bigquery --quiet
!pip install langchain==0.0.229

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sqlalchemy-bigquery 1.10.0 requires sqlalchemy<2.0.0dev,>=1.2.0, but you have sqlalchemy 2.0.0 which is incompatible.
^C


In [ ]:
!pip3 install -q -U bitsandbytes
!pip3 install -q -U peft
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 67.0 MB/s eta 0:00:00


---

#### ⚠️ Do not forget to "RESTART RUNTIME"

---

# load big query

In [ ]:

project_id = "{your_google_project}"
location = "asia-northeast3"
dataset_id = 'NAVER'
table_name = 'AD'


In [ ]:
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd "/content/drive/MyDrive/Colab Notebooks/시멘틱서치"

/content/drive/MyDrive/Colab Notebooks/시멘틱서치


In [ ]:
service_account_file = "./{your_file}.json"

In [ ]:
service_account_file

'./first-project-394906-98976fac569b.json'

In [ ]:
table_uri = f"bigquery://{project_id}/{dataset_id}"
engine = create_engine(f"bigquery://{project_id}/{dataset_id}?credentials_path={service_account_file}")

In [ ]:
query=f"""SELECT * FROM `{project_id}.{dataset_id}.{table_name}`"""
engine.execute(query).first()

<ipython-input-8-c2f5a6a300c8>:2: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  engine.execute(query).first()


(0, '2024-02', '크리마', 'year_month', '2023-02', 1260.0, 1220.0, datetime.datetime(2024, 2, 16, 16, 42, 50), datetime.datetime(2024, 2, 16, 16, 42, 50))

In [ ]:
from sqlalchemy import MetaData
metadata = MetaData()

In [ ]:
some_table = Table(table_name, metadata, autoload_with=engine)
some_table

Table('AD', MetaData(), Column('ID', Integer(), table=<AD>, nullable=False), Column('YEAR_MONTH', String(), table=<AD>, nullable=False), Column('KEYWORD', String(), table=<AD>, nullable=False), Column('CLASS', String(), table=<AD>), Column('COL', String(), table=<AD>), Column('DESKTOP', Float(), table=<AD>), Column('MOBILE', Float(), table=<AD>), Column('CREATED_AT', DATETIME(), table=<AD>), Column('UPDATED_AT', DATETIME(), table=<AD>), schema=None)

In [ ]:
import torch
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
hf_auth = userdata.get('HF_TOKEN')
model_id = "google/gemma-7b-it"
# model_id = "google/gemma-7b"
# model_id = "google/gemma-2b-it"
# model_id = "google/gemma-2b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)



In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_id,  quantization_config=bnb_config, device_map={"":0},use_auth_token = hf_auth)
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True,use_auth_token = hf_auth)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:466: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:720: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

In [ ]:
prompt = """You are a BigQuery SQL expert.
Given an input question,
create a syntactically correct BigQuery query to run

Use the following format:
Question: "Question here"
SQLQuery: "SQL Query to run"

Only use the following tables:
{table_info}

Question: Count total number of row""".format(table_info=some_table)

In [ ]:

inp = tokenizer(prompt,max_length=500, return_tensors="pt")['input_ids']
attn = tokenizer(prompt,max_length=500, return_tensors="pt")['attention_mask']
# model(inp, attn)

out = tokenizer.decode(model.generate(inp, attention_mask=attn,max_length=80, do_sample=False)[0])
out = out.replace(prompt,"")
out = out.replace("<bos><eos> in AD table\nSQLQuery: ","")
out

'SELECT COUNT(*) FROM AD'

In [ ]:
result = engine.execute(out)

for v in result:
	print(v)

(42,)
